In [ ]:
import pandas as pd

: 

In [ ]:
df = pd.read_csv('caso.csv')

In [ ]:
import csv
import gzip
import io
import json
from urllib.parse import urlencode, urljoin
from urllib.request import Request, urlopen


class BrasilIO:

    base_url = "https://api.brasil.io/v1/"

    def __init__(self, user_agent=None, auth_token=None):
        """
        Caso queria fazer uma requisição na API, passe os parâmetros user_agent e auth_token.
        Para fazer somente o download do arquivo completo, não é necessário passar nenhum parâmetro.
        """
        self.__user_agent = user_agent
        self.__auth_token = auth_token

    def headers(self, api=True):
        if api:
            return {
                "User-Agent": f"{self.__user_agent}",
                "Authorization": f"Token {self.__auth_token}"
            }
        else:
            return {
                "User-Agent": "python-urllib/brasilio-client-0.1.0",
            }
            

    def api_request(self, path, query_string=None):
        url = urljoin(self.base_url, path)

        if query_string:
            url += "?" + query_string

        request = Request(url, headers=self.headers(api=True))

        response = urlopen(request)
        return json.load(response)
        
    def data(self, dataset_slug, table_name, filters=None):
        url = f"dataset/{dataset_slug}/{table_name}/data/"
        filters = filters or {}
        filters["page"] = 1

        finished = False
        while not finished:
            query_string = "&".join([f"{k}={v}" for k, v in filters.items()])
            response = self.api_request(url, query_string)
            next_page = response.get("next", None)
            for row in response["results"]:
                yield row
            filters = {}
            url = next_page
            finished = next_page is None

    def download(self, dataset, table_name):
        url = f"https://data.brasil.io/dataset/{dataset}/{table_name}.csv.gz"
        request = Request(url, headers=self.headers(api=False))
        response = urlopen(request)
        return response


if __name__ == "__main__":
    # Caso não tenha, cadastre-se no Brasil.io e gere seu Token
    # Para mais instruções: https://blog.brasil.io/2020/10/10/como-acessar-os-dados-do-brasil-io/
    user_agent = "arthurmessias@msn.com"
    auth_token = "3f417054f93e2e4fc5a48851e0e5cc879b2f6efa"

    api = BrasilIO(user_agent, auth_token)

    dataset_slug = "covid19"
    table_name = "caso_full"

    # Para baixar o arquivo completo:

    # Após fazer o download, você salvá-lo no disco ou percorrer o arquivo em
    # memória. Para salvá-lo no disco:
    response = api.download(dataset_slug, table_name)
    with open(f"{dataset_slug}_{table_name}.csv.gz", mode="wb") as fobj:
        fobj.write(response.read())
        # TODO: o código acima pode ser melhorado de forma a não utilizar
        # `response.read()` para não colocar todo oarquivo em memória e sim fazer
        # streaming da resposta HTTP e salvar cada chunk diretamente no `fobj`.
        
    # Caso queira percorrer o CSV em memória:
    response = api.download(dataset_slug, table_name)
    fobj = io.TextIOWrapper(gzip.GzipFile(fileobj=response), encoding="utf-8")
    reader = csv.DictReader(fobj)
    for row in reader:
        pass  # faça algo com `row`
    
    # Para navegar pela API:
    filters = {"state": "PR", "is_last": True}
    data = api.data(dataset_slug, table_name, filters)
    for row in data:
        display(row)  # faça algo com `row`

{'city': None,
 'city_ibge_code': 41,
 'date': '2022-01-23',
 'epidemiological_week': 202204,
 'estimated_population': 11516840,
 'estimated_population_2019': 11433957,
 'is_last': True,
 'is_repeated': False,
 'last_available_confirmed': 1815771,
 'last_available_confirmed_per_100k_inhabitants': 15766.2258,
 'last_available_date': '2022-01-23',
 'last_available_death_rate': 0.0226,
 'last_available_deaths': 40988,
 'new_confirmed': 11753,
 'new_deaths': 6,
 'order_for_place': 683,
 'place_type': 'state',
 'state': 'PR'}

{'city': 'Abatiá',
 'city_ibge_code': 4100103,
 'date': '2021-10-26',
 'epidemiological_week': 202143,
 'estimated_population': 7408,
 'estimated_population_2019': 7457,
 'is_last': True,
 'is_repeated': False,
 'last_available_confirmed': 1138,
 'last_available_confirmed_per_100k_inhabitants': 15361.77106,
 'last_available_date': '2021-10-26',
 'last_available_death_rate': 0.0185,
 'last_available_deaths': 21,
 'new_confirmed': 0,
 'new_deaths': 0,
 'order_for_place': 537,
 'place_type': 'city',
 'state': 'PR'}

{'city': 'Adrianópolis',
 'city_ibge_code': 4100202,
 'date': '2021-10-26',
 'epidemiological_week': 202143,
 'estimated_population': 5857,
 'estimated_population_2019': 5919,
 'is_last': True,
 'is_repeated': False,
 'last_available_confirmed': 1421,
 'last_available_confirmed_per_100k_inhabitants': 24261.56736,
 'last_available_date': '2021-10-26',
 'last_available_death_rate': 0.0204,
 'last_available_deaths': 29,
 'new_confirmed': 0,
 'new_deaths': 0,
 'order_for_place': 503,
 'place_type': 'city',
 'state': 'PR'}

{'city': 'Agudos do Sul',
 'city_ibge_code': 4100301,
 'date': '2021-10-26',
 'epidemiological_week': 202143,
 'estimated_population': 9470,
 'estimated_population_2019': 9371,
 'is_last': True,
 'is_repeated': False,
 'last_available_confirmed': 850,
 'last_available_confirmed_per_100k_inhabitants': 8975.71278,
 'last_available_date': '2021-10-26',
 'last_available_death_rate': 0.0388,
 'last_available_deaths': 33,
 'new_confirmed': 2,
 'new_deaths': 0,
 'order_for_place': 536,
 'place_type': 'city',
 'state': 'PR'}

{'city': 'Almirante Tamandaré',
 'city_ibge_code': 4100400,
 'date': '2021-10-26',
 'epidemiological_week': 202143,
 'estimated_population': 120041,
 'estimated_population_2019': 118623,
 'is_last': True,
 'is_repeated': False,
 'last_available_confirmed': 11359,
 'last_available_confirmed_per_100k_inhabitants': 9462.60028,
 'last_available_date': '2021-10-26',
 'last_available_death_rate': 0.0386,
 'last_available_deaths': 439,
 'new_confirmed': 3,
 'new_deaths': 1,
 'order_for_place': 576,
 'place_type': 'city',
 'state': 'PR'}

...

{'city': 'Verê',
 'city_ibge_code': 4128609,
 'date': '2021-10-26',
 'epidemiological_week': 202143,
 'estimated_population': 7174,
 'estimated_population_2019': 7257,
 'is_last': True,
 'is_repeated': False,
 'last_available_confirmed': 1164,
 'last_available_confirmed_per_100k_inhabitants': 16225.25788,
 'last_available_date': '2021-10-26',
 'last_available_death_rate': 0.0198,
 'last_available_deaths': 23,
 'new_confirmed': 0,
 'new_deaths': 0,
 'order_for_place': 575,
 'place_type': 'city',
 'state': 'PR'}

{'city': 'Virmond',
 'city_ibge_code': 4128658,
 'date': '2021-10-26',
 'epidemiological_week': 202143,
 'estimated_population': 4022,
 'estimated_population_2019': 4023,
 'is_last': True,
 'is_repeated': False,
 'last_available_confirmed': 374,
 'last_available_confirmed_per_100k_inhabitants': 9298.85629,
 'last_available_date': '2021-10-26',
 'last_available_death_rate': 0.016,
 'last_available_deaths': 6,
 'new_confirmed': 2,
 'new_deaths': 0,
 'order_for_place': 507,
 'place_type': 'city',
 'state': 'PR'}

{'city': 'Vitorino',
 'city_ibge_code': 4128708,
 'date': '2021-10-26',
 'epidemiological_week': 202143,
 'estimated_population': 6859,
 'estimated_population_2019': 6838,
 'is_last': True,
 'is_repeated': False,
 'last_available_confirmed': 1110,
 'last_available_confirmed_per_100k_inhabitants': 16183.11707,
 'last_available_date': '2021-10-26',
 'last_available_death_rate': 0.0333,
 'last_available_deaths': 37,
 'new_confirmed': 1,
 'new_deaths': 0,
 'order_for_place': 568,
 'place_type': 'city',
 'state': 'PR'}

{'city': 'Wenceslau Braz',
 'city_ibge_code': 4128500,
 'date': '2021-10-26',
 'epidemiological_week': 202143,
 'estimated_population': 19386,
 'estimated_population_2019': 19414,
 'is_last': True,
 'is_repeated': False,
 'last_available_confirmed': 2310,
 'last_available_confirmed_per_100k_inhabitants': 11915.81554,
 'last_available_date': '2021-10-26',
 'last_available_death_rate': 0.0368,
 'last_available_deaths': 85,
 'new_confirmed': 0,
 'new_deaths': 0,
 'order_for_place': 533,
 'place_type': 'city',
 'state': 'PR'}

{'city': 'Xambrê',
 'city_ibge_code': 4128807,
 'date': '2021-10-26',
 'epidemiological_week': 202143,
 'estimated_population': 5630,
 'estimated_population_2019': 5679,
 'is_last': True,
 'is_repeated': False,
 'last_available_confirmed': 616,
 'last_available_confirmed_per_100k_inhabitants': 10941.38544,
 'last_available_date': '2021-10-26',
 'last_available_death_rate': 0.013,
 'last_available_deaths': 8,
 'new_confirmed': 1,
 'new_deaths': 0,
 'order_for_place': 477,
 'place_type': 'city',
 'state': 'PR'}